In [ ]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

In [ ]:
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = nfl.import_pbp_data([2024])
df

In [ ]:
game = df.loc[df['game_id']=="2024_21_WAS_PHI"].copy()
# game.drop(game[game['posteam'].isna()].index, inplace=True)
# game.drop(game[game['epa'].isna()].index, inplace=True)
columns = ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']
game[columns]

In [ ]:
game.loc[game['play_type']=='extra_point', columns]

In [ ]:
def game_epa(game, penalties=False):
    # remove posteam None, epa NaN. should catch start, end, timeouts
    game.drop(game[game['posteam'].isna()].index, inplace=True)
    game.drop(game[game['epa'].isna()].index, inplace=True)

    # we can't really properly measure impact of penalties, because we can't calculate expected points of the counterfactual
    # so let's just try removing plays with penalties for now. they are a part of a team's skill, so we can try adding them back in later
    # actually let's make it an arg
    if not penalties:
        game.drop(game[game['penalty']==1].index, inplace=True)

    home_team = game['home_team'].values[0]
    away_team = game['away_team'].values[0]

    df = game.groupby(['posteam', 'play_type']).agg({'epa': sum})
    home_epa = df.loc[home_team].sub(df.loc[away_team], fill_value=0) # .sub is like subtracting, but with the option to fill missing data needed for alignment between the data frames before computation
    home_epa.rename(columns={'epa': home_team}, inplace=True)
    away_epa = df.loc[away_team].sub(df.loc[home_team], fill_value=0)
    away_epa.rename(columns={'epa': away_team}, inplace=True)
    epa = pd.concat([home_epa, away_epa], axis=1) # figure out how to use keys, names, levels to get the naming correct. epa at the top of the hierarchy, then team names for the ind. columns. instead of the renaming

    return(epa)    

In [ ]:
epa = game_epa(game)
epa

In [ ]:
epa['PHI'].sum()

In [ ]:
epa['WAS'].sum()

In [ ]:
# this is a test. but what is this?